In [ ]:
from tqdm import tqdm
import argparse
from utils import *
from freebase import *
from propagation import *
import random



parser = argparse.ArgumentParser()
parser.add_argument("--dataset", type=str,
                    default="webqsp", help="choose the dataset from {webqsp, cwq}.")
parser.add_argument("--max_length", type=int,
                    default=1024, help="the max length of LLMs output.")
parser.add_argument("--temperature", type=float,
                    default=0., help="the temperature")
parser.add_argument("--llm", type=str,
                    default="llama", help="choose base LLM model from {llama, gpt-3.5-turbo, gpt-4}.")
parser.add_argument("--openai_api_key", type=str,
                    default="", help="if the LLM is gpt-3.5-turbo or gpt-4, you need add your own openai api key.")
args = parser.parse_args("")


datas, question_string = prepare_dataset(args.dataset)

In [ ]:
relations = []
for data in tqdm(datas):
    relation = data['Parses'][0]['InferentialChain']
    if relation is not None:
        relations += relation
relations = set(relations)

In [ ]:
sparql_all_relations = """
PREFIX ns: <http://rdf.freebase.com/ns/>
SELECT DISTINCT ?r
WHERE {
?e ?r ?e1 .
}
"""
r = execute_sparql(sparql_all_relations)
r = set([i['r']['value'].replace("http://rdf.freebase.com/ns/", "") for i in r if i['r']['value'].startswith("http://rdf.freebase.com/ns/")])

In [ ]:
filter_relations = list(r - relations)

In [ ]:
set([i.split('.')[-1] for i in r]) - set([i.split('.')[-1]  for i in relations])

In [ ]:
[i for i in r if 'alias' in i]